We may want to check with the user if the iteration loop should continue, it will not always be up to the agent.

In [5]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [ ]:
import json

import httpx
from agno.db.sqlite import SqliteDb
from agno.tools import tool
from agno.utils import pprint
from rich.console import Console
from rich.prompt import Prompt

console = Console()


@tool(requires_confirmation=True)
def get_top_hackernews_stories(num_stories: int) -> str:
    """Fetch top stories from Hacker News.

    Args:
        num_stories (int): Number of stories to retrieve

    Returns:
        str: JSON string containing story details
    """
    # Fetch top story IDs
    response = httpx.get("https://hacker-news.firebaseio.com/v0/topstories.json")
    story_ids = response.json()

    # Yield story details
    all_stories = []
    for story_id in story_ids[:num_stories]:
        story_response = httpx.get(
            f"https://hacker-news.firebaseio.com/v0/item/{story_id}.json"
        )
        story = story_response.json()
        if "text" in story:
            story.pop("text", None)
        all_stories.append(story)
    return json.dumps(all_stories)

In [ ]:
from agno.agent import Agent
from agno.models.google import Gemini

# Create an Agent
agent = Agent(
    model=Gemini(id="gemini-2.5-flash", temperature=0),
    instructions="You have tech-savvy personality",
    tools=[get_top_hackernews_stories],
    markdown=True,
    db=SqliteDb(session_table="test_session", db_file="tmp/example.db"),
)

In [12]:
run_response = agent.run("Fetch the top 2 hackernews stories.")
if run_response.is_paused:
    for tool in run_response.tools_requiring_confirmation:
        # Ask for confirmation
        console.print(
            f"Tool name [bold blue]{tool.tool_name}({tool.tool_args})[/] requires confirmation."
        )
        message = (
            Prompt.ask("Do you want to continue?", choices=["y", "n"], default="y")
            .strip()
            .lower()
        )

        if message == "n":
            tool.confirmed = False
        else:
            # We update the tools in place
            tool.confirmed = True

run_response = agent.continue_run(run_response=run_response)
# Or
# run_response = agent.continue_run(run_id=run_response.run_id, updated_tools=run_response.tools)

pprint.pprint_run_response(run_response)

INFO Successfully created table 'test_session'

Tool name get_top_hackernews_stories({'num_stories': 2}) requires confirmation.

Do you want to continue? [y/n] (y):

╭──────────────────────────────────────────────────────────────────╮
│ Here are the top 2 Hacker News stories:                          │
│                                                                  │
│ 1.  **Kagi News**                                                │
│     *   URL: https://blog.kagi.com/kagi-news                     │
│     *   Score: 396                                               │
│     *   By: grappler                                             │
│                                                                  │
│ 2.  **Launch HN: Airweave (YC X25) – Let agents search any app** │
│     *   URL: https://github.com/airweave-ai/airweave             │
│     *   Score: 61                                                │
│     *   By: lennertjansen                                        │
╰──────────────────────────────────────────────────────────────────╯